# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
CLIENT_PORT = 62012
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", CLIENT_PORT)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Sun 2023-01-22 10:27:51.588702 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
Sun 2023-01-22 10:27:51.589496 [INFO ] - [T] Attempting to bind to 0.0.0.0:62012
Sun 2023-01-22 10:27:51.589513 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Sun 2023-01-22 10:27:51.590266 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Sun 2023-01-22 10:27:51.627762 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f9293a5e150>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
CRESET    = '\33[0m'
CBOLD     = '\33[1m'
CITALIC   = '\33[3m'
CURL      = '\33[4m'
CBLINK    = '\33[5m'
CBLINK2   = '\33[6m'
CSELECTED = '\33[7m'

CBLACK  = '\33[30m'
CRED    = '\33[31m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CBLUE   = '\33[34m'
CVIOLET = '\33[35m'
CBEIGE  = '\33[36m'
CWHITE  = '\33[37m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    else:
        color = CVIOLET
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
client_tiface

server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
server_tiface

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

(client_fiface, server_fiface)

(<FilterInterface at 0x7f92900b4c20>, <FilterInterface at 0x7f92900b4fb0>)

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient299", client_version: "0.0.666" }

In [11]:
jupyter_endpoint = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("sendrequestaction", endpoint=jupyter_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }

In [12]:
await client_fiface.command_response(filter_cmd)

Sun 2023-01-22 10:27:51.808786 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
LTI(client)5 skipped 4: received transport_cmd NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }


Accepted

Sun 2023-01-22 10:27:51.809123 [TRACE] - [T<-F,C] Processing command NewEndpoint { endpoint: Endpoint(127.0.0.1:62013), timeout: 5s }


In [13]:
# XXX get endpoint programatically from client
client_endpoint = EndpointW(f"127.0.0.1:{CLIENT_PORT}")
client_endpoint

Endpoint(127.0.0.1:62012)

In [15]:
server_notifications = server_fiface.get_notifications()
assert len(server_notifications) > 0
new_request_action = server_notifications[0]

assert new_request_action.request_action == connect_action

In [16]:
cookie = base64.urlsafe_b64encode(bytes(new_request_action.request_action.name.encode()))
cookie

Sun 2023-01-22 10:27:55.181546 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
Sun 2023-01-22 10:27:55.193782 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }


b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='

Sun 2023-01-22 10:27:55.441309 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
Sun 2023-01-22 10:27:55.442413 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }


In [17]:
response_ok = ResponseCodeW("loggedin", cookie=str(cookie), server_version="0.0.666")
response_ok

Sun 2023-01-22 10:27:55.701087 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
Sun 2023-01-22 10:27:55.721786 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }


ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }

In [18]:
await server_fiface.command_response(FilterCmdW("sendresponsecode", endpoint=client_endpoint, code=response_ok))

Sun 2023-01-22 10:27:55.948782 [TRACE] - [F<-A,C] New command: SendResponseCode { endpoint: Endpoint(127.0.0.1:62012), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }
LTI(server)140 skipped 129: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }] }


Accepted

Sun 2023-01-22 10:27:55.949653 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62012), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 9999s }], packets: [Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }] }
Sun 2023-01-22 10:27:55.949708 [TRACE] - [T<-UDP] Response { sequence: 1, request_ack: Some(1), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }


In [19]:
client_notifications = client_fiface.get_notifications()
assert len(client_notifications) > 0
new_response_code = client_notifications[0]
new_response_code

LTI(server)141: got transport_rsp back from Transport layer: TransportRsp::Accepted
Sun 2023-01-22 10:27:55.953063 [TRACE] - [F<-T,R] Command Accepted
Sun 2023-01-22 10:27:55.961223 [TRACE] - [T<-UDP] Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
LTI(server)142: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62012), packet: Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } } }]
Sun 2023-01-22 10:27:55.970570 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient299", client_version: "0.0.666" } }
LTI(server)143: got transport notifications: []
Sun 2023-01-22 10:27:56.043704 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62013), Took packet Respo

NewResponseCode { endpoint: Endpoint(127.0.0.1:62013), code: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }

LTI(client)53: got transport notifications: []


In [20]:
new_response_code.response_code

ResponseCodeW { inner: LoggedIn { cookie: "b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='", server_version: "0.0.666" } }

In [21]:
cookie_from_server = new_response_code.response_code.cookie
assert str(cookie_from_server) == str(cookie)

### Convenience function

In [22]:
async def request_action_to_filter(req_action, endpoint=jupyter_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await client_fiface.command_response(filter_cmd)

### Join a room

In [23]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Sun 2023-01-22 10:27:57.338896 [TRACE] - [F<-A,C] New command: SendRequestAction { endpoint: Endpoint(127.0.0.1:62013), action: JoinRoom { room_name: "general" } }
LTI(client)65 skipped 11: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }] }
Sun 2023-01-22 10:27:57.339826 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }] }


Accepted

Sun 2023-01-22 10:27:57.339897 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
LTI(client)66: got transport_rsp back from Transport layer: TransportRsp::Accepted
Sun 2023-01-22 10:27:57.343321 [TRACE] - [F<-T,R] Command Accepted
LTI(client)67: got transport notifications: []
Sun 2023-01-22 10:27:57.369957 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
LTI(server)188 skipped 44: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62012), tid: ProcessUniqueId { prefix: 0, offset: 1 } }
Sun 2023-01-22 10:27:57.370160 [TRACE] - [T<-F,C] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62012), tid: ProcessUniqueId { prefix: 0, offset: 1 } }
LTI(server)189: got transport_rsp back from Transp

In [24]:
client_fiface.get_notifications()

Sun 2023-01-22 10:27:57.591787 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:27:57.619223 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }


[]

Sun 2023-01-22 10:27:57.850933 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:27:57.868195 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:27:58.101628 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:27:58.115908 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: JoinRoom { room_name: "general" } }
Sun 2023-01-22 10:27:58.361125 [TRACE] - [T<-UDP] Request { sequence: 2, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90

LTI(client)103: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)104: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 13, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
Sun 2023-01-22 10:27:58.974620 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 2, offset: 4 }, retry_interval: 250ms }], packets: [Request { sequence: 13, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
L

LTI(client)115: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Sun 2023-01-22 10:27:59.176495 [TRACE] - [F<-T,N] For Endpoint Endpoint(127.0.0.1:62012), Took packet Request { sequence: 5, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }
LTI(client)116: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 9 }, retry_interval: 250ms }], packets: [Request { sequence: 18, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
Sun 2023-01-22 10:27:59.177070 [TRACE] - [T<-F,C] Processing comman

...

LTI(client)197: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)198: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 30 }, retry_interval: 250ms }], packets: [Request { sequence: 54, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
Sun 2023-01-22 10:28:00.189989 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId

LTI(client)221: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Sun 2023-01-22 10:28:00.391615 [TRACE] - [T<-UDP] Request { sequence: 43, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }
Sun 2023-01-22 10:28:00.391633 [TRACE] - [T<-UDP] Request { sequence: 44, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }
Sun 2023-01-22 10:28:00.391640 [TRACE] - [T<-UDP] Request { seq

LTI(client)249: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)250: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 42 }, retry_interval: 250ms }], packets: [Request { sequence: 78, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
LTI(client)251: got transport_rsp back from Transport layer: TransportRsp::AcceptedSun 2023-01-22 10:28:00.594908 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: Pro

LTI(client)273: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
LTI(client)274: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 47 }, retry_interval: 250ms }], packets: [Request { sequence: 89, response_ack: Some(1), cookie: Some("b'RmlsdGVyTm90ZWJvb2tDbGllbnQyOTk='"), action: KeepAlive { latest_response_ack: 1 } }] }
Sun 2023-01-22 10:28:00.797006 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 47 }, retry_interval: 250ms }], packets: [Request

LTI(client)297: got transport notifications: [TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointTimeout { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:62013) }]
Sun 2023-01-22 10:28:00.999160 [INFO ] - [F<-T,N] Endpoint Endpoint(127.0.0.1:62013) timed-out. Dropping.
Sun 2023-01-22 10:28:00.999169 [WARN ] - [F] error sending KeepAlive during idle endpoint (Endpoint(127.0.0.1:62013)): Filter does not contain an entry for the endpoint: Endpoint(127.0.0.1:62013)
Sun 2023-01-22 10:28:00.999174 [WARN ] - [F] error sending KeepAlive during idle endpoint (Endpoint(127.0.0.1:62013)): Filter does not contain an entry 